## **Connect to Google Drive**

In [ ]:
!pip install gdown

In [ ]:
# 224 x 224
# !gdown --folder 'https://drive.google.com/drive/folders/1r-VVeY47jW8O78oebrzzn_b8KhCX4HVC?usp=share_link'

# 28 x 28
!gdown --folder 'https://drive.google.com/drive/folders/1bmlGhMrsL-911W9sOivk_2URdW8zp2hm?usp=sharing'

In [ ]:
%cd /kaggle/working/npy_files
# %cd /kaggle/working/224_224_gray_invert_dilate
%ls

## **Load Data**

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
def get_mini_batch(input_X, label, batch_size):
    for i in range(0, len(input_X), batch_size):
        yield input_X[i: i + batch_size], label[i: i + batch_size]

def get_mini(input_X, batch_size):
    for i in range(0, len(input_X), batch_size):
        yield input_X[i: i + batch_size]

In [ ]:
def load_images_from_npy():
    covid_images = np.load('./covid.npy', allow_pickle=True)
    hb_images = np.load('./hb.npy', allow_pickle=True)
    mi_images = np.load('./mi.npy', allow_pickle=True)
    normal_images = np.load('./normal.npy', allow_pickle=True)
    pmi_images = np.load('./pmi.npy', allow_pickle=True)


    # covid - 0
    # hb - 1
    # mi - 2
    # normal - 3
    # pmi - 4

    # fill labels 0 for covid, 1 for hb, 2 for mi, 3 for normal, 4 for pmi
    labels = [0] * (len(covid_images) // 1)
    labels.extend([1] * (len(hb_images) // 1))
    labels.extend([2] * (len(mi_images) // 1))
    labels.extend([3] * (len(normal_images) // 1))
    labels.extend([4] * (len(pmi_images) // 1))

    images = np.concatenate((covid_images, hb_images, mi_images, normal_images, pmi_images), axis=0)

    images = np.array(images)
    labels = np.array(labels)

    return images, labels



def combine_leads_of_image(data):
  
    # store each 13 lead images in a list of 13*28*28 arrays
    images = []

    for X in get_mini(data, 13):
        images.append(X)

    images = np.array(images)
    return images

In [ ]:
images, labels = load_images_from_npy()
print(images.shape, labels.shape, images[0].shape)

images = combine_leads_of_image(images)
print(images.shape)

In [ ]:
def stack_images(images):
    stacked_images = np.empty((images.shape[0],images.shape[1]*images.shape[2],images.shape[3]))

    for i in range(images.shape[0]):
        stacked_parts = np.vstack(images[i])
        stacked_images[i] = stacked_parts
    
    return stacked_images

In [ ]:
def train_val_split(images, labels, split_factor=0.8, shuffle=True):
    # if shuffle:
    random_indices = np.random.choice(len(images), len(images), replace=False)
    training_data_full = images[random_indices]
    training_label_full = labels[random_indices]

    split_index_train = int(split_factor * len(training_data_full))
    split_index_test = int(0.9 * len(training_data_full))

    training_data = training_data_full[:split_index_train]
    training_label = training_label_full[:split_index_train]

    validation_data = training_data_full[split_index_train : split_index_test]
    validation_label = training_label_full[split_index_train : split_index_test]

    test_data = training_data_full[split_index_test :]
    test_label = training_label_full[split_index_test :]

    return training_data, training_label, validation_data, validation_label, test_data, test_label

## **Build Model**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torch.optim.lr_scheduler as lr_scheduler

In [ ]:
class LSTMNetwork(nn.Module):
    def __init__(self):
        super(LSTMNetwork, self).__init__()

        self.flatten = nn.Flatten(start_dim=2, end_dim=3)

        # output of the flatten layer = output
        # lstm input_size must match output.shape[-1]
        self.lstm = nn.LSTM(input_size=784, hidden_size=10, num_layers=1, batch_first=True, 
                                dropout=0.25, bidirectional=True)
        
        self.classification_layers = nn.Sequential(
            nn.Flatten(start_dim=1),
            nn.LazyLinear(64),
            nn.ReLU(),
            nn.LazyLinear(5),
            nn.LogSoftmax(dim=1)
        )


    def forward(self, input_X):
        
        input_X = self.flatten(input_X)
        input_X, (h, c) = self.lstm(input_X)
        
        for layer in self.classification_layers:
            input_X = layer(input_X)

        return input_X

In [67]:
class HybridNetwork(nn.Module):
    def __init__(self):
        super(HybridNetwork, self).__init__()

        # build LeNet-5 using nn.Sequential
        self.extraction_layers = nn.Sequential(
            nn.LazyConv2d(out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.LazyConv2d(out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Flatten(start_dim=1, end_dim=2),
        )
        
        self.lstm = nn.LSTM(input_size=7, hidden_size=10, num_layers=1, batch_first=True, 
                                dropout=0.25, bidirectional=False)
        
        self.classification_layers = nn.Sequential(
            nn.Flatten(start_dim=1),
            nn.LazyLinear(64),
            nn.ReLU(),
            nn.LazyLinear(5),
            nn.LogSoftmax(dim=1)
        )


    def forward(self, input_X):

        for layer in self.extraction_layers:
            input_X = layer(input_X)
        
        input_X, (h, c) = self.lstm(input_X)
        
        for layer in self.classification_layers:
            input_X = layer(input_X)

        return input_X

In [ ]:
class Model:
    def __init__(self):
        self.network = HybridNetwork()
#         self.network = LSTMNetwork()
        
        self.num_epochs = 10
        self.batch_size = 16
        self.grad_clipping = 10.0
        self.optimizer_type = 'adamax'
        self.lr = 0.001
        self.num_output_units = 5

        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.network.to(self.device)    

    print('Device:', self.device)  

        self.init_optimizer()

    def init_optimizer(self):
        parameters = [p for p in self.network.parameters() if p.requires_grad]
        if self.optimizer_type == 'sgd':
            self.optimizer = optim.SGD(parameters, self.lr,
                                        momentum=0.4,
                                        weight_decay=0)
        elif self.optimizer_type == 'adamax':
            self.optimizer = optim.Adamax(parameters,
                                        lr=self.lr,
                                        weight_decay=0)
        else:
            raise RuntimeError('Unsupported optimizer: %s' %
                                self.optimizer_type)
        self.scheduler = lr_scheduler.MultiStepLR(self.optimizer, milestones=[10, 15], gamma=0.5)


    def normalize(self, X):
        # apply standard normalization on x. mean = 0, std = 1
        X = (X - np.mean(X)) / np.std(X)

        return X


    def reshape_image(self, input_X, label):
        
        # shape of input_X = (batch_size, 13, 28, 28)
        X_full = input_X.reshape(input_X.shape[0], input_X.shape[1], input_X.shape[2], input_X.shape[3])

        ''' shape of input_X = (batch_size, 13*28, 28)
        X_full = input_X.reshape(input_X.shape[0], 1, input_X.shape[1], input_X.shape[2])
        '''
        y_full = np.eye(self.num_output_units)[label.astype(int)]

        return X_full, y_full

    def report_num_trainable_parameters(self):
        num_parameters = 0
        for p in self.network.parameters():
            if p.requires_grad:
                sz = list(p.size())

                num_parameters += np.prod(sz)
        print('Number of parameters: ', num_parameters)


    def predict(self, test_data, test_label):
        self.network.eval()

        X = test_data
        y = test_label

        X = self.normalize(X)
        cnt = 0

        for X_batch, y_batch in tqdm(get_mini_batch(input_X=X, label=y, batch_size=self.batch_size), desc="Testing"):

            # reshape to appropriate format
            X, y_true = self.reshape_image(X_batch, y_batch)

            # convert to float32 and convert to torch tensor
            X = X.astype(np.float32)
            y_true = y_true.astype(np.float32)
            X = torch.from_numpy(X)
            y_true = torch.from_numpy(y_true)
            
            X = X.to(self.device)
            y_true = y_true.to(self.device)

            pred_proba = self.network(X)

            y_pred = torch.argmax(pred_proba, dim=1)
            y_true = torch.argmax(y_true, dim=1)
            
            print(y_true, y_pred)

            cnt += torch.sum(y_pred == y_true).item()

        print(f'accuracy: {cnt/test_data.shape[0]}')


    def evaluate(self, validation_data, validation_label):
        self.network.eval()

        X = validation_data
        y = validation_label

        X = self.normalize(X)
        cnt = 0

        for X_batch, y_batch in tqdm(get_mini_batch(input_X=X, label=y, batch_size=self.batch_size), desc="Evaluating"):

            # reshape to appropriate format
            X, y_true = self.reshape_image(X_batch, y_batch)

            # convert to float32 and convert to torch tensor
            X = X.astype(np.float32)
            y_true = y_true.astype(np.float32)
            X = torch.from_numpy(X)
            y_true = torch.from_numpy(y_true)

            X = X.to(self.device)
            y_true = y_true.to(self.device)

            pred_proba = self.network(X)

            y_pred = torch.argmax(pred_proba, dim=1)
            y_true = torch.argmax(y_true, dim=1)

            cnt += torch.sum(y_pred == y_true).item()

        print(f'accuracy: {cnt/validation_data.shape[0]}')


    def train(self, input_X, label):
        self.network.train()
        
        self.updates = 0
        iter_cnt, num_iter = 0, (len(input_X) + self.batch_size - 1) // self.batch_size

        for X_batch, y_batch in tqdm(get_mini_batch(input_X=input_X, label=label, batch_size=self.batch_size), desc="Training"):

            # reshape to appropriate format
            X, y_true = self.reshape_image(X_batch, y_batch)

            # convert to float32 and convert to torch tensor
            X = X.astype(np.float32)
            y_true = y_true.astype(np.float32)
            X = torch.from_numpy(X)
            y_true = torch.from_numpy(y_true)
            
            X = X.to(self.device)
            y_true = y_true.to(self.device)

            pred_proba = self.network(X)

            loss = F.cross_entropy(pred_proba, y_true)
            self.optimizer.zero_grad()
            loss.backward()

            torch.nn.utils.clip_grad_norm_(self.network.parameters(), self.grad_clipping)

            # Update parameters
            self.optimizer.step()

            self.updates += 1
            iter_cnt += 1

#             if self.updates % 1000 == 0:
#                 print('Iter: %d/%d, Loss: %f' % (iter_cnt, num_iter, loss.item()))
        
        print('Iter: %d/%d, Loss: %f' % (iter_cnt, num_iter, loss.item()))
        self.scheduler.step()
        print('LR:', self.scheduler.get_last_lr()[0])            


    def fit(self, training_data, training_label, validation_data, validation_label):

        X = training_data
        y = training_label

        X = self.normalize(X)

        for epoch in range(self.num_epochs):
            self.train(input_X=X, label=y)
            print("Epoch: ", epoch)

            self.evaluate(validation_data, validation_label)

In [ ]:
def main():
    training_data, training_label, validation_data, validation_label, test_data, test_label = train_val_split(images=images, labels=labels, split_factor=0.8, shuffle=True)
    print(training_data.shape, training_label.shape, validation_data.shape, validation_label.shape, test_data.shape, test_label.shape)
   
    model = Model()

    model.evaluate(validation_data, validation_label)
    model.fit(training_data, training_label, validation_data, validation_label)

    model.predict(test_data, test_label)

main()

(28558, 28, 28) (28558,) (3570, 28, 28) (3570,) (3570, 28, 28) (3570,)
Use cuda: True


Evaluating: 224it [00:02, 89.72it/s]


accuracy: 0.10504201680672269


Training: 1785it [00:44, 40.06it/s]


Iter: 1785/1785, Loss: 0.546465
LR: 0.001
Epoch:  0


...

tensor([0, 1, 1, 4, 3, 3, 1, 4, 3, 3, 3, 3, 3, 2, 2, 1], device='cuda:0') tensor([0, 1, 3, 4, 3, 3, 1, 4, 3, 1, 3, 3, 3, 2, 2, 1], device='cuda:0')
tensor([3, 4, 1, 1, 3, 3, 1, 3, 3, 3, 3, 1, 1, 1, 1, 1], device='cuda:0') tensor([3, 4, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 1, 2, 1, 1], device='cuda:0')
tensor([3, 3, 1, 3, 1, 3, 4, 3, 4, 2, 2, 1, 1, 1, 3, 3], device='cuda:0') tensor([3, 3, 1, 3, 1, 3, 2, 3, 4, 2, 2, 1, 3, 1, 3, 3], device='cuda:0')


Testing: 207it [00:02, 84.46it/s]

tensor([3, 1, 4, 3, 1, 1, 3, 0, 1, 3, 3, 2, 2, 3, 3, 3], device='cuda:0') tensor([1, 1, 4, 3, 1, 2, 3, 0, 1, 1, 1, 2, 2, 3, 1, 3], device='cuda:0')
tensor([3, 0, 3, 3, 2, 2, 0, 1, 0, 4, 0, 3, 0, 1, 3, 1], device='cuda:0') tensor([3, 0, 3, 3, 2, 2, 0, 1, 4, 4, 0, 1, 0, 1, 3, 3], device='cuda:0')
tensor([1, 3, 0, 1, 3, 1, 3, 1, 4, 1, 3, 2, 1, 3, 2, 3], device='cuda:0') tensor([1, 1, 0, 1, 3, 1, 3, 1, 4, 1, 3, 2, 1, 1, 2, 3], device='cuda:0')
tensor([1, 1, 4, 0, 4, 3, 0, 3, 3, 3, 3, 4, 4, 3, 3, 3], device='cuda:0') tensor([1, 1, 4, 0, 4, 1, 0, 3, 3, 3, 3, 2, 4, 3, 3, 3], device='cuda:0')
tensor([2, 4, 1, 2, 3, 1, 1, 3, 1, 2, 3, 2, 3, 1, 3, 4], device='cuda:0') tensor([2, 4, 1, 2, 3, 1, 1, 1, 1, 2, 3, 2, 3, 1, 3, 4], device='cuda:0')
tensor([3, 2, 3, 3, 0, 3, 3, 3, 3, 3, 2, 0, 3, 3, 3, 3], device='cuda:0') tensor([3, 2, 3, 3, 0, 3, 3, 3, 3, 3, 2, 0, 3, 1, 3, 3], device='cuda:0')
tensor([4, 2, 2, 2, 2, 2, 0, 1, 4, 2, 1, 1, 0, 3, 3, 0], device='cuda:0') tensor([4, 2, 2, 2, 2, 2, 0, 3, 4, 2, 

Testing: 224it [00:02, 86.93it/s]

tensor([1, 1, 1, 1, 1, 0, 3, 1, 1, 3, 3, 2, 3, 2, 3, 0], device='cuda:0') tensor([1, 1, 1, 1, 1, 0, 1, 1, 1, 3, 3, 2, 3, 2, 3, 0], device='cuda:0')
tensor([3, 1, 4, 4, 4, 2, 4, 0, 2, 3, 3, 1, 1, 0, 3, 1], device='cuda:0') tensor([3, 1, 4, 4, 4, 2, 4, 0, 1, 3, 3, 1, 1, 0, 3, 1], device='cuda:0')
tensor([3, 3, 0, 3, 2, 1, 3, 3, 3, 3, 1, 3, 2, 2, 4, 3], device='cuda:0') tensor([3, 3, 0, 3, 2, 1, 3, 3, 3, 3, 1, 1, 2, 2, 4, 3], device='cuda:0')
tensor([3, 3, 3, 2, 3, 3, 0, 0, 3, 4, 1, 2, 2, 1, 3, 3], device='cuda:0') tensor([3, 3, 3, 2, 3, 3, 0, 0, 3, 4, 1, 1, 2, 1, 3, 3], device='cuda:0')
tensor([3, 1, 4, 3, 1, 3, 3, 3, 1, 1, 3, 4, 4, 3, 2, 2], device='cuda:0') tensor([3, 1, 4, 1, 1, 3, 3, 3, 3, 1, 3, 4, 4, 3, 2, 1], device='cuda:0')
tensor([1, 2, 4, 3, 0, 1, 1, 1, 3, 4, 0, 2, 3, 0, 3, 0], device='cuda:0') tensor([1, 2, 4, 3, 0, 3, 1, 1, 3, 4, 0, 2, 3, 0, 3, 0], device='cuda:0')
tensor([4, 3, 3, 3, 3, 1, 1, 1, 1, 2, 1, 4, 3, 2, 1, 2], device='cuda:0') tensor([0, 3, 3, 3, 3, 1, 1, 1, 3, 2, 